In [1]:
import numpy as np

In [2]:
    n = 2
    m = 12

    lx = ly = 750000
    x = np.linspace(-lx, lx, n)
    y = np.linspace(-ly, ly, n)
    t = (np.arange(12) + 0.5) / 12

    # assign temperature and precipitation values
    (yy, xx) = np.meshgrid(y, x)
    temp = np.zeros((m, n, n))
    prec = np.zeros((m, n, n))
    stdv = np.zeros((m, n, n))
    for i in range(len(t)):
        temp[i] = -10 * yy / ly - 5 * np.cos(i * 2 * np.pi / 12)
        prec[i] = xx / lx * (np.sign(xx) - np.cos(i * 2 * np.pi / 12))
        stdv[i] = (2 + xx / lx - yy / ly) * (1 - np.cos(i * 2 * np.pi / 12))

    T_obs = temp.reshape(m, -1)
    P_obs = prec.reshape(m, -1)
    S_obs = stdv.reshape(m, -1)

    temp_snow=0.0
    temp_rain=2.0


In [3]:
    def f_inst_pdd(temp, stdv):
        """Compute instantaneous positive degree days from temperature.

        Use near-surface air temperature and standard deviation to compute
        instantaneous positive degree days (effective temperature for melt,
        unit degrees C) using an integral formulation (Calov and Greve, 2005).

        *temp*: array_like
            Near-surface air temperature in degrees Celcius.
        *stdv*: array_like
            Standard deviation of near-surface air temperature in Kelvin.
        """
        import scipy.special as sp

        # compute positive part of temperature everywhere
        positivepart = np.greater(temp, 0) * temp

        # compute Calov and Greve (2005) integrand, ignoring division by zero
        with np.errstate(divide="ignore", invalid="ignore"):
            normtemp = temp / (np.sqrt(2) * stdv)
        calovgreve = stdv / np.sqrt(2 * np.pi) * np.exp(
            -(normtemp**2)
        ) + temp / 2 * sp.erfc(-normtemp)

        # use positive part where sigma is zero and Calov and Greve elsewhere
        teff = np.where(stdv == 0.0, positivepart, calovgreve)

        # convert to degree-days
        return teff * 365.242198781

    def f_accu_rate(temp, prec):
        """Compute accumulation rate from temperature and precipitation.

        The fraction of precipitation that falls as snow decreases linearly
        from one to zero between temperature thresholds defined by the
        `temp_snow` and `temp_rain` attributes.

        *temp*: array_like
            Near-surface air temperature in degrees Celcius.
        *prec*: array_like
            Precipitation rate in meter per year.
        """

        # compute snow fraction as a function of temperature
        reduced_temp = (temp_rain - temp) / (temp_rain - temp_snow)
        snowfrac = np.clip(reduced_temp, 0, 1)

        # return accumulation rate
        return snowfrac * prec

    def f_melt_rates(snow, pdd):
        """Compute melt rates from snow precipitation and pdd sum.

        Snow melt is computed from the number of positive degree days (*pdd*)
        and the `pdd_factor_snow` model attribute. If all snow is melted and
        some energy (PDD) remains, ice melt is computed using `pdd_factor_ice`.

        *snow*: array_like
            Snow precipitation rate.
        *pdd*: array_like
            Number of positive degree days.
        """

        # parse model parameters for readability
        ddf_snow = 3 / 1000
        ddf_ice = 8 / 1000

        # compute a potential snow melt
        pot_snow_melt = ddf_snow * pdd

        # effective snow melt can't exceed amount of snow
        snow_melt = np.minimum(snow, pot_snow_melt)

        # ice melt is proportional to excess snow melt
        ice_melt = (pot_snow_melt - snow_melt) * ddf_ice / ddf_snow

        # return melt rates
        return (snow_melt, ice_melt)


In [4]:
        temp = T_obs
        prec = P_obs
        stdv = S_obs
        # compute accumulation and pdd
        accu_rate = f_accu_rate(temp, prec)
        inst_pdd = f_inst_pdd(temp, stdv)

        # initialize snow depth, melt and refreeze rates
        snow_depth = np.zeros_like(temp)
        snow_melt_rate = np.zeros_like(temp)
        ice_melt_rate = np.zeros_like(temp)
        snow_refreeze_rate = np.zeros_like(temp)
        ice_refreeze_rate = np.zeros_like(temp)


In [5]:
temp

array([[  5.        , -15.        ,   5.        , -15.        ],
       [  5.66987298, -14.33012702,   5.66987298, -14.33012702],
       [  7.5       , -12.5       ,   7.5       , -12.5       ],
       [ 10.        , -10.        ,  10.        , -10.        ],
       [ 12.5       ,  -7.5       ,  12.5       ,  -7.5       ],
       [ 14.33012702,  -5.66987298,  14.33012702,  -5.66987298],
       [ 15.        ,  -5.        ,  15.        ,  -5.        ],
       [ 14.33012702,  -5.66987298,  14.33012702,  -5.66987298],
       [ 12.5       ,  -7.5       ,  12.5       ,  -7.5       ],
       [ 10.        , -10.        ,  10.        , -10.        ],
       [  7.5       , -12.5       ,   7.5       , -12.5       ],
       [  5.66987298, -14.33012702,   5.66987298, -14.33012702]])

In [6]:
for i in range(len(temp)):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11


In [7]:
sd_changes = np.array([(snow_depth[i-1]+accu_rate[i])-f_melt_rates(snow_depth[i-1]+accu_rate[i],inst_pdd[i])[0] for i in range(len(temp))])

In [8]:
np.array([sd_changes[0:i+1].sum(0) for i in range(len(temp)) if i > 0])

array([[ 0.        ,  3.8660254 ,  0.        ,  0.1339746 ],
       [ 0.        ,  5.3660254 ,  0.        ,  0.6339746 ],
       [ 0.        ,  6.3660254 ,  0.        ,  1.63397448],
       [ 0.        ,  6.8660254 ,  0.        ,  3.12738652],
       [ 0.        ,  7.        ,  0.        ,  4.87872607],
       [ 0.        ,  7.        ,  0.        ,  6.65700856],
       [ 0.        ,  7.1339746 ,  0.        ,  8.4083481 ],
       [ 0.        ,  7.6339746 ,  0.        ,  9.90176014],
       [ 0.        ,  8.6339746 ,  0.        , 10.90176003],
       [ 0.        , 10.1339746 ,  0.        , 11.40176003],
       [ 0.        , 12.        ,  0.        , 11.53573462]])

In [9]:
        # compute snow depth and melt rates
        for i in range(len(temp)):
            if i > 0:
                snow_depth[i] = snow_depth[i - 1]
            snow_depth[i] = snow_depth[i] + accu_rate[i]
            snow_melt_rate[i], ice_melt_rate[i] = f_melt_rates(snow_depth[i], inst_pdd[i])
            snow_depth[i] -= snow_melt_rate[i]
print(snow_depth)

[[ 0.          2.          0.          0.        ]
 [ 0.          3.8660254   0.          0.1339746 ]
 [ 0.          5.3660254   0.          0.6339746 ]
 [ 0.          6.3660254   0.          1.63397448]
 [ 0.          6.8660254   0.          3.12738652]
 [ 0.          7.          0.          4.87872607]
 [ 0.          7.          0.          6.65700856]
 [ 0.          7.1339746   0.          8.4083481 ]
 [ 0.          7.6339746   0.          9.90176014]
 [ 0.          8.6339746   0.         10.90176003]
 [ 0.         10.1339746   0.         11.40176003]
 [ 0.         12.          0.         11.53573462]]
